**完了確認済み2020/07/24**

# 不等式のQUBO変換を用いて基底エネルギー以下のサンプルを得る


#### **このファイルのミッション**
今までの誤解を取り払って、条件を整えて分析する


#### **データ、条件**
small, binary data

### **調整したこと**
- XはSNPを想定。SNPの発生する箇所は95%ほとんどある塩基、5%ある別の塩基としか出現しないので0, 1で置換可能。
- Xは標準化しない
- 特徴量選択は内積が大きいものを取り出す
- ay-byは内積のままでok
- 場合によってはイジングモデルを採用しよう

##### import

In [38]:
import itertools
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_regression
import random
from pyqubo import Array, OneHotEncInteger, solve_qubo
import scipy.stats

#### X

In [39]:
#標準偏差が0である列をなくす
def drop_str0_X(X):
    for each_col in X.columns:
        if X[each_col].std()==0.0:
            X = X.drop(each_col, axis=1)
    return X

In [40]:
X_ori55 = pd.read_csv("../input/SNP_df55.csv", sep=',', index_col=0)
X_ori55_ad = drop_str0_X(X_ori55)

In [41]:
max_X_element = X_ori55_ad.max(axis=1).max() 
min_X_element = X_ori55_ad.min(axis=1).min() 

num_samples = X_ori55_ad.shape[0]
num_cols = X_ori55_ad.shape[1]

#### y

In [42]:
ori_y5 = pd.read_csv('../input/y5.csv', sep=',', index_col=0)['y']
ori_y5

0    1
1    0
2    1
3    1
4    0
Name: y, dtype: int64

In [43]:
selected_col_num1 = 1
selected_col_num2 = 2
selected_col_num3 = 3

#### 特徴量選択

In [44]:
def featrure_InnerProduct_sel_mul(X, y, sel_col_num):
    inner_product_list = [np.dot(X[each_col], y) for each_col in X.columns] #内積のリスト
    sort_inner_product_list = sorted(inner_product_list ,reverse=True)
    
    select_cols = []
    for i in range(sel_col_num):
        iindex = inner_product_list.index(sort_inner_product_list[i])
        select_cols.append(X.columns[iindex])
    
    else_cols = list(X.columns)
    for col in select_cols:
        else_cols.remove(col)
    
    return select_cols, else_cols

In [45]:
select_cols, else_cols = featrure_InnerProduct_sel_mul(X_ori55_ad, ori_y5, selected_col_num2)

In [47]:
select_cols
else_cols

['C', 'D', 'E']

### ハミルトニアン
したいこと→ハミルトニアンを最小化させる（xを最適化）

In [48]:
# W : ay - byの最大値

#xの要素で最大値とxの要素の最小値を知れば推定はできる
W = max_X_element *1*num_samples - min_X_element*1*num_samples

In [49]:
# while True & if break バージョン
def make_y(y_after):
    while True:
        select_index = random.randrange(num_samples)
        y_after[select_index] = 1 - y_after[select_index]
        if np.dot(np.array([1]*num_samples).T, y_after) == sum(list(ori_y)):
            if y_after.std():
                #print(y_after.std())
                break
    return y_after 

In [50]:
def make_ax_by(X, y, select_cols, else_cols):
    ij = []
    ax_by = []
    for i in select_cols:
        for j in else_cols:
            row_each_axby = np.dot(X[i], y) - np.dot(X[j], y)
            ax_by.append(row_each_axby)
    return ax_by

In [63]:
def H(ax_by, index_ax_by):
    # W : ay - byの最大値
    #W = X.max(axis=1).max() *1*num_samples - X.min(axis=1).min()*1*num_samples
    
    z = [0]*W
        
    each_axby = ax_by[index_ax_by]
    sum_w_z = 0
    
    if each_axby >= 0: #変えた
        z[each_axby] = 1
        sum_w_z = sum([each_axby*z[w] for w in range(W)])
    else:
        z[0] = 1
        sum_w_z = sum([1*z[w] for w in range(W)])
        
    sum_z = sum(z)

    ans_h =  (1-sum_z)**2 + (sum_w_z - each_axby)**2
    return ans_h

In [64]:
def H_sum(ax_by):
    H_sum =  sum([H(ax_by, index_ax_by) for index_ax_by in range(len(ax_by))])
    return H_sum

In [65]:
ax_by = make_ax_by(X_ori55_ad, ori_y5, select_cols, else_cols)

In [66]:
#特徴量を複数選ぶときは、どうしても0となってしまう箇所が含まれる確率が高くなる
#エラーではない
ax_by

[1, 1, 1, 0, 0, 0]

In [67]:
ori_H_sum = H_sum(ax_by)
ori_H_sum

0

In [68]:
#ori_yからはめでたいことに一つも出力されなくなった
for index_ax_by in range(len(ax_by)):
    if H(ax_by, index_ax_by) > 0:
        print(H(ax_by, index_ax_by)) #Hの値を知る
        #print(ij[index_ax_by]) #どの特徴量同士から発生している問題か知る
        print(ax_by[index_ax_by]) #マイナスになっているのか確認
        print('--------')